In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../src")

In [4]:
import mesh
from tqdm.autonotebook import tqdm

/Users/seantyh/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
import pandas as pd
from CwnGraph import CwnBase, CwnLemma

In [6]:
meld_sch=pd.read_excel(mesh.get_data_dir()/"ldt-data/Tsang-2018-MELD-SCH.xlsx")

In [7]:
clpw = pd.read_excel(mesh.get_data_dir()/"ldt-data/Tse-2017-Chinese-lexicon-project.xlsx", sheet_name=0)

In [8]:
cwn = CwnBase()

In [9]:
# get lemmas with at least one sense
lemma_iter = filter(lambda x: x[1]['node_type']=='lemma', cwn.V.items())
lemma_iter = filter(lambda x: CwnLemma(x[0], cwn).senses, lemma_iter)
lemma_iter = map(lambda x: x[1].get("lemma"), lemma_iter)

cwn_lemmas = list(lemma_iter)

## CLD Dataset

In [17]:
cld = pd.read_csv(mesh.get_data_dir()/"CLD/chineselexicaldatabase2.1.txt")

/Users/seantyh/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,4,8,9,12,13,17,18,22,23,28,29,111,112,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
from opencc import OpenCC
s2t = OpenCC('s2t')
cld=cld.assign(trad_word=lambda df: [s2t.convert(x) for x in df.Word])

In [49]:
sub_cld = cld[["trad_word", "FrequencyRaw"]]

## Graph measures

In [29]:
cns = pd.read_csv(mesh.get_data_dir()/"cns/cwn_centralities.csv", index_col=0)

In [30]:
cns.head()

,word,nsenses,degree,closeness,betweenness,eigen,ncompo
0,60,0,1,0.000012,0.0,0.000000e+00,21
1,68,0,1,0.000012,0.0,0.000000e+00,21
2,9.20%,1,1,0.000012,0.0,0.000000e+00,21
3,BJ,0,1,0.000012,0.0,0.000000e+00,10
4,GG,1,1,0.000021,0.0,2.368389e-10,35392


## MELD_SCH & CWN coverage

In [67]:
cn_words = meld_sch.loc[meld_sch.lexicality==1,].word.tolist()

In [68]:
from opencc import OpenCC
cc = OpenCC('s2t')

In [69]:
tw_words = [cc.convert(x) for x in cn_words]

In [81]:
int_words = set(cwn_lemmas) & set(tw_words)
print(
    f"MELD_SCH Words: {len(cn_words)}\n"
    f"Intersect with CWN: {len(int_words)}")

MELD_SCH Words: 12578
Intersect with CWN: 2931


In [82]:
meld_sch = meld_sch.assign(trad_word=lambda df: [s2t.convert(x) for x in df.word])

In [83]:
meld_sch_words = pd.DataFrame(data={"word":list(int_words)})

In [84]:
meld_sch.head()

,id,word,lexicality,N,RT,RTSD,zRT,zRTSD,ERR,length,...,C3pr2,C3pr3,C3pr4,C3pr5,C4pr1,C4pr2,C4pr3,C4pr4,C4pr5,trad_word
0,100001,乜,1,24,1180.965833,350.387139,0.875071,0.914858,40.000000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,乜
1,100002,凡,1,40,778.381250,319.598542,-0.135300,1.137753,4.761905,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,凡
2,100003,口,1,41,682.423659,223.998070,-0.593711,0.682099,2.380952,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,口
3,100004,土,1,40,704.935000,129.813901,-0.439805,0.509694,0.000000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,土
4,100005,女,1,40,680.626500,240.384837,-0.561245,0.738961,0.000000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,女


In [89]:
meld_sch_frame = meld_sch_words.merge(meld_sch.loc[meld_sch.lexicality==1,:][["trad_word", "N", "RT", "RTSD", "ERR", "length"]], 
                                    left_on="word", right_on="trad_word")
meld_sch_frame = meld_sch_frame.merge(cns)
# meld_sch_frame = meld_sch_frame.merge(sub_cld, left_on="word", right_on="trad_word")

In [90]:
meld_sch_words.shape

(2931, 1)

In [91]:
meld_sch_frame.tail()

,word,trad_word,N,RT,RTSD,ERR,length,nsenses,degree,closeness,betweenness,eigen,ncompo
2929,歷史,歷史,40,620.990000,261.371009,0.000000,2,9,1,0.000021,0.000000,1.553057e-09,35392
2930,無恥,無恥,39,688.402564,275.194471,2.500000,2,1,1,0.000012,0.000000,0.000000e+00,6
2931,地球,地球,39,605.464872,174.062097,2.500000,2,2,1,0.000021,0.000000,7.229293e-07,35392
2932,歸,歸,40,691.740500,305.748072,4.761905,1,11,2,0.000021,349727.296041,1.822807e-04,35392
2933,可喜,可喜,38,796.014211,272.666809,7.317073,2,1,1,0.000021,0.000000,6.961061e-12,35392


In [92]:
meld_sch_frame.to_csv(mesh.get_data_dir()/"cns/cns_meld_sch.csv", encoding="UTF-8")

## CLP & CWN Coverage

In [14]:
clp_words = clpw.Word_Trad.tolist()

In [15]:
int_words = set(cwn_lemmas) & set(clp_words)
print(
    f"CLP Words: {len(clp_words)}\n"
    f"Intersect with CWN: {len(int_words)}")

CLP Words: 25281
Intersect with CWN: 4128
